# Data Loading & Processing

In [40]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import pandas as pd
from useful_tools import UsefulTools

# LOAD DATA
training_data = pd.DataFrame(UsefulTools.JsonCache.load('../../data/train_data_15_50.json', expected_type=list))
test_data = pd.DataFrame(UsefulTools.JsonCache.load('../../data/test_data_10_50.json', expected_type=list))


# PROCESS DATA

Loaded cache from '../../data/train_data_15_50.json' (66818 items).
Loaded cache from '../../data/test_data_10_50.json' (32453 items).


In [41]:
# Count sentences per source
source_counts = training_data['source'].value_counts()

# Get the top 4 sources with highest sentence count
top_sources = source_counts.head(4).index

# Filter both datasets to include only these top 4 sources
training_data = training_data[training_data['source'].isin(top_sources)].reset_index(drop=True)
test_data = test_data[test_data['source'].isin(top_sources)].reset_index(drop=True)

print(f"\nSelected top sources: {list(top_sources)}")
print(f"Training data (n sentences):\n{training_data[['id','source']].groupby('source').count().reset_index().sort_values(by='id',ascending=False).values}")
print(f"\nTest data (n sentences):\n{test_data[['id','source']].groupby('source').count().reset_index().sort_values(by='id',ascending=False).values}")
print(f"\nTraining sources: {training_data.source.unique()}")
print(f"Test sources: {test_data.source.unique()}")
print(f"Training samples: {len(training_data)}")
print(f"Test samples: {len(test_data)}")


Selected top sources: ['SiliconANGLE News', 'Forbes', 'The Verge', 'Dwarkesh.com']
Training data (n sentences):
[['SiliconANGLE News' 7015]
 ['Forbes' 5889]
 ['The Verge' 3831]
 ['Dwarkesh.com' 3562]]

Test data (n sentences):
[['Forbes' 5653]
 ['The Verge' 1629]]

Training sources: ['The Verge' 'Forbes' 'SiliconANGLE News' 'Dwarkesh.com']
Test sources: ['The Verge' 'Forbes']
Training samples: 20297
Test samples: 7282


In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from useful_tools import UsefulTools

# 1. Preprocessing
full_train_texts = training_data['sentence'].tolist() + test_data['sentence'].tolist()
full_train_labels = training_data['source'].tolist() + test_data['source'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in training_data['source']]
y_dev = [label2id[label] for label in test_data['source']]

# 2. TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 6),
    max_features=50000
)

tfidf.fit(full_train_texts)
X_train = tfidf.transform(training_data['sentence'])
X_dev = tfidf.transform(test_data['sentence'])

# 3. Hyperparameter Tuning with CVGridSearch
param_grid = {
    'C': [0.1, 0.5, 1.0, 1.5, 2.0, 5],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7]
}
fixed_params = {
    'penalty': 'elasticnet',
    'solver': 'saga',
    'max_iter': 5000,
    'random_state': 42,
    'n_jobs': -1,
    'class_weight': 'balanced'
}

model = LogisticRegression(**fixed_params)
halving_search = UsefulTools.HalvingGridSearch(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=3,
    factor=2,
    cache_file='./cache/halving_logreg.pkl',
    verbose=1
)

print("Running cross-validation grid search...")
best_model, best_params, best_score = halving_search.search_and_fit(X_train, y_train)


# 4. Final Model Training
enet = LogisticRegression(**fixed_params, **best_params)
print("Fitting best model...")
enet.fit(X_train, y_train)

# 5. Evaluation
print("Evaluating model on dev set...")
dev_preds = enet.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score: {dev_macro_f1:.4f}")


Running cross-validation grid search...
🔁 Loaded HalvingGridSearchCV from cache: ./cache/halving_logreg.pkl
🏆 Best Params: {'C': 2.0, 'l1_ratio': 0.3}
📈 Best Score: 0.5378
Fitting best model...
Evaluating model on dev set...
Validation Macro F1 Score: 0.3929


In [43]:
# 5. Evaluation
X_dev = tfidf.transform(test_data['sentence'])
print("Evaluating model on dev set...")
dev_preds = enet.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score: {dev_macro_f1:.4f}")

Evaluating model on dev set...
Validation Macro F1 Score: 0.3929


In [49]:
def predict_custom_sentence(sentence: str, top_k: int = 4):
    # Vectorize the input sentence
    vectorized = tfidf.transform([sentence])

    # Predict class ID and get probabilities
    probas = enet.predict_proba(vectorized)[0]
    pred_id = np.argmax(probas)
    pred_label = id2label[pred_id]

    # Sort and show top_k class probabilities
    top_indices = np.argsort(probas)[::-1][:top_k]
    # print(f"\nInput Sentence: {sentence}")
    print(f"Predicted Source: {pred_label}")
    print("\nTop Predictions:")
    for idx in top_indices:
        print(f"  {id2label[idx]:<15} : {probas[idx]:.4f}")

    return pred_label


# Example usage
sample = "Making sure that a user is who they say they are is at the core of San Francisco-based Persona, which offers ID authentication software to 3,000 companies including OpenAI, LinkedIn, Etsy, Reddit, DoorDash and Robinhood."
predict_custom_sentence(sample)

Predicted Source: The Verge

Top Predictions:
  The Verge       : 0.4796
  Forbes          : 0.3074
  Dwarkesh.com    : 0.1162
  SiliconANGLE News : 0.0967


'The Verge'

# roBERTa

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # <-- Correct import here!
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np
import os

# PyTorch-ready dataset
class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts              # Store the list of comment texts
        self.labels = labels            # Store the corresponding list of integer labels
        self.tokenizer = tokenizer      # Store the tokenizer (e.g., RobertaTokenizer)
        self.max_length = max_length    # Store the maximum sequence length

    def __len__(self):
        return len(self.texts)  # Important: this allows DataLoader to know dataset size

    def __getitem__(self, idx):
        """
        Fetches a single sample from the dataset.

        Args:
            idx (int): Index of the item.

        Returns:
            dict: A dictionary containing:
                - 'input_ids': Raw text must be converted into numbers because models can't understand plain words.
                - 'attention_mask': A list that tells the model which tokens are real (1) and which ones are just padding (0).
                - 'labels': The label

        * Tensor is a multi-dimensional array (like a super-powered NumPy array) that can run on a GPU very efficiently

        ex:
        Raw Text:
        "eating soap to own the republicans"

        Tokenized to IDs:
        [0, 1553, 4153, 7, 1225, 5, 13815, 2, *1, *1]

        Attention Mask:
        [1, 1, 1, 1, 1, 1, 1, 1, *0, *0]

        Label:
        0

        Models usually expect inputs to have the same length (max = max_lenght).
        If your token list is too short, you add padding tokens (which are just zeros).
        """
        # Fetch the text and its corresponding label using the index
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text:
        # - truncation: cuts off texts longer than max_length
        # - padding: adds padding tokens to shorter texts
        # - return_tensors='pt': returns PyTorch tensors instead of lists
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length = self.max_length,
            return_tensors='pt' # stands for PyTorch
        )

        # Return a dictionary with model inputs and label
        return {
            'input_ids': encoding['input_ids'].squeeze(),         # Tensor of input token IDs .squeeze() again just removes the extra dimension.
            'attention_mask': encoding['attention_mask'].squeeze(), # Tensor indicating real tokens vs padding
            'labels': torch.tensor(label, dtype=torch.long)        # Make sure it becomes a PyTorch Tensor, because the model expects labels to be Tensors during training.
        }
    

from torch.amp import GradScaler, autocast
# Use smaller (half-size) numbers for most calculations, so training is much faster and uses less memory — without losing much accuracy.
scaler = GradScaler()

# Mixed Precision Training
def train_MPT(model, loader):
    model.train()
    total_loss = 0
    for batch in tqdm(loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with autocast(device_type=device.type):  # 🔥 autocast for mixed precision
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation Function
def evaluate(model, loader):
    model.eval() # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad(): # Turns off gradient calculations (Saves memory and speeds up evaluation because we don't need gradients)
        for batch in tqdm(loader):
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Get predictions (pick the class with the highest score (probability) for each example in the batch)
            preds = torch.argmax(outputs.logits, dim=1) # outputs.logits = raw scores for each class
            # Save predictions and true labels
            # We Move predictions and labels from GPU to CPU and convert them into numpy arrays
            # because some Python operations (like NumPy or scikit-learn) cannot work directly with GPU Tensors.
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Take the F1 score for each label (class) separately, and then average them equally (It does not care if one label has 1000 examples and another class has 100 examples)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return macro_f1

In [ ]:
# 1. Preprocessing
full_train_texts = training_data['sentence'].tolist() + test_data['sentence'].tolist()
full_train_labels = training_data['source'].tolist() + test_data['source'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in training_data['source']]
y_dev = [label2id[label] for label in test_data['source']]

# 2. TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 6),
    max_features=50000
)

tfidf.fit(full_train_texts)
X_train = tfidf.transform(training_data['sentence'])
X_dev = tfidf.transform(test_data['sentence'])

MODEL_NAME = 'roberta-base'
MAX_LENGTH = 128
BATCH_SIZE = 64
EPOCHS = 2
LEARNING_RATE = 2e-5

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

training_data['label_id'] = training_data['source'].map(label2id)
test_data['label_id'] = test_data['source'].map(label2id)

train_dataset = RedditDataset(training_data['sentence'].tolist(), training_data['label_id'].tolist(), tokenizer, MAX_LENGTH)
dev_dataset = RedditDataset(test_data['sentence'].tolist(), test_data['label_id'].tolist(), tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE)

# Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda is faster but it requires GPU
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
model.to(device) # Moves all model weights and computations to the device you selected (cuda if GPU, otherwise cpu)

# Optimizer and Scheduler
# model.parameters() are all the learnable weights in the model.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE) # A type of optimizer that updates the model's weights to minimize the loss. 
total_steps = len(train_loader) * EPOCHS 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps) # It controls how the learning rate changes during training. In our case it linearly decreases 


MODEL_SAVE_PATH = f'cache/best_{MODEL_NAME}_model_{MAX_LENGTH}-{BATCH_SIZE}-{EPOCHS}-{LEARNING_RATE}.pt'

# Check if saved model exists
if os.path.exists(MODEL_SAVE_PATH):
    print(f"Loading saved model from {MODEL_SAVE_PATH}...")
    model.load_state_dict(torch.load(MODEL_SAVE_PATH))
else:
    print("No saved model found. Starting training...")

    # Run Training
    best_f1 = 0
    best_model_state = None

    for epoch in range(EPOCHS): # One epoch = One full pass through the entire training dataset.
        print(f"Epoch {epoch+1}/{EPOCHS}") 
        train_loss = train_MPT(model, train_loader)
        val_f1 = evaluate(model, dev_loader)
        print(f"Train Loss: {train_loss:.4f}, Validation Macro F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    print(f'Best f1 for this model: {best_f1}')

    # After training, save the best model
    torch.save(best_model_state, MODEL_SAVE_PATH)
    print(f"Training finished. Best model saved to {MODEL_SAVE_PATH}")

    # Loads back the best version of the model that you saved during training (Forget your current weights — load these saved weights instead)
    model.load_state_dict(best_model_state) 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Bohua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Bohua\AppData\Local\Temp\ipykernel_2644\4112795864.py", line 19, in <module>
    tfidf.fit(full_train_texts)
  File "C:\Users\Bohua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Bohua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py", line 2074, in fit
    X = super().fit_transform(raw_documents)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

In [ ]:
def predict_sentence_roberta(text):
    model.eval()
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu().numpy().flatten()
        pred_idx = np.argmax(probs)
        print(f"\n📝 Sentence: {text}")
        print(f"📢 Predicted Source: {id2label[pred_idx]}")
        print("\n🔍 Top Probabilities:")
        for idx in probs.argsort()[::-1]:
            print(f"  {id2label[idx]:<15} : {probs[idx]:.4f}")